In [3]:
%%capture
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset Preprocessing

In [4]:
#Importing the data

train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
submission= pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [5]:
print('Number of train samples: ', train.shape[0])
print('Number of test samples: ', test.shape[0])

Number of train samples:  3662
Number of test samples:  1928


In [8]:
train["id_code"] = train["id_code"].apply(lambda x: x + ".png")
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")
train['diagnosis'] = train['diagnosis'].astype('str')
train.head()

,id_code,diagnosis
0,000c1434d8d7.png,2
1,001639a390f0.png,4
2,0024cdab0c1e.png,1
3,002c21358ce6.png,0
4,005b95c28852.png,0


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

train_datagen=ImageDataGenerator(rescale=1./255,horizontal_flip=True)


train_generator_aptos=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",
    target_size=(224, 224),
    subset='training')





'''valid_generator_aptos = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",    
    target_size=(224, 224),
    subset='validation')
'''


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator_aptos = test_datagen.flow_from_dataframe(  
        dataframe=test,
        directory = "../input/aptos2019-blindness-detection/test_images/",
        x_col="id_code",
        target_size=(224, 224),
        batch_size=16,
        shuffle=False,
        class_mode=None)

Found 3662 validated image filenames belonging to 5 classes.
Found 1928 validated image filenames.


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

# Define paths to your dataset folders
train_data_dir = '/kaggle/input/cataractdataset/dataset'
#validation_data_dir = '/path/to/your/dataset/validation'

# Create ImageDataGenerator instances for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,validation_split=0.2)
#validation_datagen = ImageDataGenerator(rescale=1./255)

# Define the batch size
batch_size = 16

# Create generators for training and validation data
train_generator_cataract = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # adjust the target size as needed
    batch_size=batch_size,
    class_mode='categorical',  # assuming you have multiple classes
    subset='training'
)



"""validation_generator_cataract = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)"""


test_generator_cataract = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 481 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [ ]:
%%capture
!pip install tensorflow-federated

In [17]:
!pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 38.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.3
    Uninstalling scipy-1.9.3:
      Successfully uninstalled scipy-1.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflow-federated 0.70.0 requires scipy~=1.9.3, but you have scipy 1.12.0 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=

In [18]:
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Define a simple model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Wrap the model with TensorFlow Federated
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(model, input_spec=train_generator_aptos.element_spec)

# Create a federated learning process
fed_process = tff.learning.build_federated_averaging_process(model_fn)

# Federated training loop
for round_num in range(10):
    # Initialize the server state
    state = fed_process.initialize()

    # Train on the two clients
    state, metrics = fed_process.next(state, [(train_generator_aptos, train_generator_cataract)])

    # Print metrics
    print(f'Round {round_num}: {metrics}')

# Evaluate the federated model
evaluation = tff.learning.build_federated_evaluation(model_fn)
test_metrics_aptos = evaluation(state.model, test_generator_aptos)
test_metrics_cataract = evaluation(state.model, test_generator_cataract)
print(f'Test Metrics Aptos: {test_metrics_aptos}')
print(f'Test Metrics Cataract: {test_metrics_cataract}')


ImportError: cannot import name '_sub_module_deprecation' from 'scipy._lib.deprecation' (/opt/conda/lib/python3.10/site-packages/scipy/_lib/deprecation.py)